In [3]:
pip install pycryptodomex --no-binary :all:

Skipping wheel build for pycryptodomex, due to binaries being disabled for it.
    Running setup.py install for pycryptodomex: started
    Running setup.py install for pycryptodomex: still running...
    Running setup.py install for pycryptodomex: finished with status 'done'
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install pycryptodome-test-vectors

  Created wheel for pycryptodome-test-vectors: filename=pycryptodome_test_vectors-1.0.11-py2.py3-none-any.whl size=12144772 sha256=a2c8a40a2cba75958219966e509754a9972bf1d5f6648b23314401d2273d2e1f
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\7e\3e\18\e05e30d26c1eccdcf60864d9adb7f13c22d7c099ad2a4ec6b3
Successfully built pycryptodome-test-vectors
Note: you may need to restart the kernel to use updated packages.


In [15]:
from Cryptodome.Cipher import AES

In [104]:
import numpy as np
import math
import Cryptodome as Crypto
from Cryptodome.PublicKey import RSA
from Cryptodome.PublicKey import DSA
from Cryptodome.PublicKey import ECC

In [2]:
r = 12
alpha = [ 0.01, 0.05, 0.1 ]
quantil = [ 2.327, 1.645 , 1.285 ]
Xi_1 = [0]*3
Xi_2 = [0]*3
Xi_3 = [0]*3
print(Xi_1)
Xi_1 = [i*np.sqrt(255 * 2) + 255 for i in quantil]

Xi_2 = [i*np.sqrt(255 * 255 * 2) + 255*255 for i in quantil]

Xi_3 = [i*np.sqrt(255 * 255 * 2) + 255*(r-1) for i in quantil]

print(Xi_3)

[0, 0, 0]
[3644.173114708759, 3398.227234076454, 3268.402429050604]


In [3]:
N = 32
Bytes = 524288

In [4]:
get_random_bytes = bytearray()
for i in range(int(Bytes/N)):
    get_random_bytes += Crypto.Random.get_random_bytes(N)

len(get_random_bytes)

524288

In [5]:
getrandbits = bytearray()
for i in range(int(Bytes)):
    getrandbits += Crypto.Random.random.getrandbits(8).to_bytes(1, 'big')

len(getrandbits)

524288

In [85]:
n = 2048
RSApk = bytearray()
for i in range(int(Bytes/(n/8))):
    a = Crypto.PublicKey.RSA.generate(n)
    RSApk += a.p.to_bytes(128, 'big') + a.q.to_bytes(128, 'big')

In [86]:
len(RSApk)

524288

In [100]:
ECCpk = bytearray()
while (len(ECCpk)<Bytes):
    ECCpk += ECC.generate(curve='P-256').d.to_bytes()
    
len(ECCpk)

524303

In [107]:
DSApk = bytearray()
while (len(DSApk)<Bytes):
    DSApk += DSA.generate(2048).p.to_bytes(256, 'big')
    
len(DSApk)

In [108]:
len(DSApk)

200960

In [7]:
def criterion_1(data):
    Xi = 0.0
    H = [True, True, True]
    n = int(len(data)/256)
    byte_count = [0]*256
    for i in range(256):
        byte_count[i] = data.count(i)
        Xi += ((byte_count[i] - n)**2 / n)
    for i in range(3):
        if Xi > Xi_1[i]:
            H[i] = False
    
    return Xi, H

In [8]:
def criterion_2(data):    
    Xi = 0.0
    H = [True, True, True]
    n = int(len(data)/2)
    byte_count = [[0]*256 for i in range(256)]
    first = [0]*256
    second = [0]*256
    for i in range(n):
        byte_count[data[2*i]][data[2*i+1]] += 1
        first[data[2*i]] += 1
        second[data[2*i+1]] += 1
        
    for i in range(256):
        for j in range(256):
            Xi += ((byte_count[i][j]**2)/(first[i]*second[j]))
    Xi = (Xi-1)*n
    for i in range(3):
        if Xi > Xi_2[i]:
            H[i] = False
    
    return Xi, H

In [9]:
def criterion_3(data):    
    Xi = 0.0
    H = [True, True, True]
    n = int(len(data))
    m_ = int((n/r))
    n = m_*r
    byte_count_r = [[0]*12 for i in range(256)]
    byte_count = [0]*256
    for j in range(r):
        for i in range(m_):
            byte_count_r[data[i + j*m_]][j] +=1
            byte_count[data[i + j*m_]] += 1
    for i in range(256):
        for j in range(12):
            Xi += ((byte_count_r[i][j]**2)/(byte_count[i]*m_))
    Xi = (Xi-1)*n
    for i in range(3):
        if Xi > Xi_3[i]:
            H[i] = False
    
    return Xi, H


In [10]:
#Maurer’s “Universal Statistical” Test
#, L, expV, var
def criterion_4(data):    
    Xi = 0.0
    sum_ = 0
    H = [True, True, True]
    data_bit = ""
    for i in data:
        data_bit += '{0:08b}'.format(i)
    n = int(len(data_bit))
    Q = 10*(2**L)
    T = [0]*(2**L)
    for i in range(Q):
        T[int(data_bit[i*L: i*L+L],2)] = i
    K = int(n/L - Q)
    for i in range(K):
        sum_ += math.log2(i+Q - T[int(data_bit[Q*L + i*L: Q*L + i*L+L],2)])
        T[int(data_bit[Q*L + i*L: Q*L + i*L+L],2)] = i+Q
    f_n = sum_/K
    c = 0.7 - (0.8/L) + (4 + 32/L)*(pow(K, (-3/L))/15)
    gamma = c*math.sqrt(var/K)
    P = math.erfc(abs(f_n - expV)/(math.sqrt(2)*gamma))
    #1% rule states to check P-value < 0.01 => False
    for i in range(3):
        if P < alpha[i]:
            H[i] = False
    return P, H


L = 8
expV = 7.1836656
var = 3.238

In [21]:
#Random Excursions Variant Test
def criterion_5(data):    
    Xi = 0.0
    sum_ = 0
    H = [True, True, True]
    data_bit = ""
    for i in data:
        data_bit += '{0:08b}'.format(i)
    n = int(len(data_bit))
    xi = [0]*19
    S = 0
    xi[9] += 1
    for i in range(n):
        if(data_bit[i] == '1'):
            S += 1
        else:
            S -= 1
        if (abs(S)<=9):
            xi[S+9] += 1
    J = xi[9]
    P = [0]*19
    
    for i in range(19):
        if(i != 9):
            P[i] = math.erfc(abs(xi[i]-J)/math.sqrt(2*J*(4*abs(i-9)-2)))
    #1% rule states to check P-value < 0.01 => False
    P.pop(9)
    for i in range(3):
        for j in range(18):
            if P[j] < alpha[i]:
                H[i] = False
                break
    return P, H


In [ ]:
print("Xi for criteria 1: " Xi_1)
print("Xi for criteria 2: " Xi_2)
print("Xi for criteria 3: " Xi_3)
print("alpha: " alpha)

In [23]:
print(criterion_1(get_random_bytes))
print(criterion_2(get_random_bytes))
print(criterion_3(get_random_bytes))
print(criterion_4(get_random_bytes))
print(criterion_5(get_random_bytes))

(231.9208984375, [True, True, True])
(64733.273770514585, [True, True, True])
(2713.2082519033584, [True, True, True])
(0.11150358912339814, [True, True, True])
([0.9397309575096909, 0.7292594547341302, 0.6972207829947468, 0.569588085563856, 0.5160423810100728, 0.3864762307712326, 0.32567103678286924, 0.2606338425315887, 0.3577648175618734, 0.9875638831765086, 0.7391579082705373, 0.944426752288836, 0.4264218854129438, 0.5261650033087717, 0.9700088087582979, 0.7457646413634914, 0.702215354184505, 0.5106732390107467], [True, True, True])


In [87]:
print(criterion_1(getrandbits))
print(criterion_2(getrandbits))
print(criterion_3(getrandbits))
print(criterion_4(getrandbits))
print(criterion_5(getrandbits))

(244.1982421875, [True, True, True])
(64698.98133374803, [True, True, True])
(2899.372616706713, [True, True, True])
(0.36201293506084126, [True, True, True])
([0.5918420734686674, 0.7198380560481024, 0.8326671994984294, 0.7355318762955201, 0.8149745964518661, 0.7433286647292777, 0.5930625981066426, 0.36064704767104244, 0.16478920460634883, 0.6326610502373972, 0.9518631313415185, 0.7233073123857566, 0.5956319201472444, 0.6940671765173658, 0.8712014685757845, 0.5959202492164613, 0.4026437860695563, 0.3670210957124318], [True, True, True])


In [88]:
print(criterion_1(RSApk))
print(criterion_2(RSApk))
print(criterion_3(RSApk))
print(criterion_4(RSApk))
print(criterion_5(RSApk))


(408.1591796875, [False, False, False])
(65215.779879146896, [True, True, True])
(2787.129886081236, [True, True, True])
(0.11066727041051792, [True, True, True])
([0.10927093392001368, 0.10598585916278155, 0.05577945284478564, 0.03495507264363056, 0.027734470235048183, 0.02529232439639864, 0.03770719138706869, 0.14981213516709155, 0.6597890810315775, 0.6955808273182371, 0.21404421720997704, 0.18937309460325985, 0.18932919450771196, 0.17083529853378324, 0.11799977503975212, 0.05753961187264973, 0.059875515962479545, 0.06595420164423037], [True, False, False])


In [101]:
print(criterion_1(ECCpk))
print(criterion_2(ECCpk))
print(criterion_3(ECCpk))
print(criterion_4(ECCpk))
print(criterion_5(ECCpk))


(238.53857421875, [True, True, True])
(64878.67444660347, [True, True, True])
(2851.986795278689, [True, True, True])
(0.06229353442169084, [True, True, False])
([0.9043132010267174, 0.5664130791782349, 0.4953312122622224, 0.4880392736138859, 0.5431833514113554, 0.8221278693532301, 0.8522892477910072, 0.8907458009320663, 0.634194817329482, 0.08103475417128944, 0.20798030941952225, 0.5764381972359925, 0.9521962019705574, 0.7560957014958838, 0.7559159758919114, 0.9473890666567608, 0.6523621476238477, 0.427535517515224], [True, True, False])


In [109]:
print(criterion_1(DSApk))
print(criterion_2(DSApk))
print(criterion_3(DSApk))
print(criterion_4(DSApk))
print(criterion_5(DSApk))

(264.0636942675158, [True, True, True])
(64817.988896884526, [True, True, True])
(2817.9734278418814, [True, True, True])
(0.5684911190508302, [True, True, True])
([0.2938128087536618, 0.5545805612380654, 0.68903734077464, 0.6635466389640836, 0.7028835190625773, 0.7068509494219011, 0.6563239748867087, 0.4551658905375948, 0.6906192201976447, 0.028590893764117786, 0.04748223864823018, 0.15445188236025, 0.21457149348404794, 0.2594429134031647, 0.24806677651344947, 0.12560788237384796, 0.06072428097795445, 0.042643136577263874], [True, False, False])
